In [2]:
import threading
import time


def task(i):
    start = time.time()
    print(f'Threading: {threading.current_thread().name} - Task{i} start!')
    #time.sleep(1)
    # 替换成计算的代码
    ans = 1
    for j in range(1,100000):
        ans = ans * j
    end = time.time()
    print(f'Threading: {threading.current_thread().name} - Task{i} cost {end-start} seconds!\n',end='')


# 串行执行三个任务
print('串行执行:')
start = time.time()
for i in range(1, 4):
    task(i)
end = time.time()
print(f'Cost {end - start} seconds!')

# 并行执行三个任务
print('多线程并行执行:')
start = time.time()
thread_list = []
for i in range(1, 4):
    t = threading.Thread(target=task, args=(i, ))
    thread_list.append(t)
    t.start()
    
for t in thread_list:
    t.join()

end = time.time()
print(f'Cost {end - start} seconds!')


串行执行:
Threading: MainThread - Task1 start!
Threading: MainThread - Task1 cost 3.0894510746002197 seconds!
Threading: MainThread - Task2 start!
Threading: MainThread - Task2 cost 3.6657679080963135 seconds!
Threading: MainThread - Task3 start!
Threading: MainThread - Task3 cost 3.1853253841400146 seconds!
Cost 9.940544366836548 seconds!
多线程并行执行:
Threading: Thread-6 (task) - Task1 start!
Threading: Thread-7 (task) - Task2 start!
Threading: Thread-8 (task) - Task3 start!
Threading: Thread-8 (task) - Task3 cost 7.931380987167358 seconds!
Threading: Thread-6 (task) - Task1 cost 9.364147663116455 seconds!
Threading: Thread-7 (task) - Task2 cost 9.484148502349854 seconds!
Cost 9.516110181808472 seconds!


In [11]:
def gen_cal():
    num_list = []
    exp = None
    while True:
        if exp is None:
            exp = yield num_list.pop(0) if num_list else None
        else:
            num_list = exp
            exp = yield max(num_list) if num_list else None

gc = gen_cal()
assert(next(gc) == None)  # 初始list为空输出None
assert(gc.send([2, 7, 4]) == 7)  # 传入[2,7,4]，输出最大值7
assert(next(gc) == 2)  # 调用next，输出第一个数2
assert(next(gc) == 7)  # 再次调用next，输出第二个数7
assert(gc.send([1, 2]) == 2)  # 传入[1,2]，输出最大值2
assert(next(gc) == 1)  # 调用next，输出第一个数1
assert(gc.send([]) == None)  # 传入空list，输出None


In [12]:
waiting_list = []
def worker1():
    # 等级1两个，等级2一个，等级3一个，等级4两个
    task = {1: 2, 2: 1, 3: 1, 4: 2}
    rank = 1
    while sum(task.values()) > 0:
        if task[rank] > 0:
            task[rank] = task[rank]-1
            print(f'Task1: level {rank} subtask remains {task[rank]}.')
            yield
        elif task[rank] == 0:
            rank += 1
    print('Task1 finished!')


def worker2():
    # 等级1两个，等级2四个，等级3三个
    task = {1: 2, 2: 4, 3: 3}
    rank = 1
    while sum(task.values()) > 0:
        if task[rank] > 0:
            task[rank] = task[rank]-1
            print(f'Task2: level {rank} subtask remains {task[rank]}.')
            yield
        elif task[rank] == 0:
            rank += 1
    print('Task2 finished!')


class Handle(object):

    def __init__(self, gen):
        self.gen = gen

    def call(self):
        next(self.gen)


def RoundRobin(*worker):
    waiting_list.extend(Handle(c) for c in worker)
    while waiting_list:
        for p in waiting_list:
            try:
                p.call()
            except StopIteration:
                waiting_list.remove(p)
    print('All Task Finished!')


w1 = worker1()
w2 = worker2()
RoundRobin(w1, w2)


Task1: level 1 subtask remains 1.
Task2: level 1 subtask remains 1.
Task1: level 1 subtask remains 0.
Task2: level 1 subtask remains 0.
Task1: level 2 subtask remains 0.
Task2: level 2 subtask remains 3.
Task1: level 3 subtask remains 0.
Task2: level 2 subtask remains 2.
Task1: level 4 subtask remains 1.
Task2: level 2 subtask remains 1.
Task1: level 4 subtask remains 0.
Task2: level 2 subtask remains 0.
Task1 finished!
Task2: level 3 subtask remains 2.
Task2: level 3 subtask remains 1.
Task2: level 3 subtask remains 0.
Task2 finished!
All Task Finished!


# 4 async/await
worker_1只能完成第一类和第三类的任务，worker_2只能完成第二类的任务，请参考课件中的例子完成以下要求：worker如果在自己的任务队列中发现自己不能完成的任务，就将其交给另一个worker去完成(只需要修改任务队列，对任务执行的顺序不做要求，不考虑一个worker执行完所有任务之后，另一个worker又将任务交给他的情况)。(3分)

In [9]:
import asyncio


class Task():
    def __init__(self, type, time):
        self.type = type  # 任务类型
        self.time = time  # 任务耗时


task_list1 = [Task(1, 2), Task(1, 3), Task(2, 2), Task(3, 4)]
task_list2 = [Task(2, 1), Task(2, 3), Task(2, 4), Task(3, 2)]
interval = .1
# 完成任务时调用working函数


async def working(worker_id, task: Task):
    """working函数

    Args:
        worker_id : worker的编号，0或1
        task : 任务对象
    """
    print(
        f"worker {worker_id} is working on task {task.type} and cost {task.time} seconds!")
    await asyncio.sleep(task.time)


async def worker_1():
    while task_list1:
        task = task_list1.pop(0)
        if task.type == 2:
            task_list2.append(task)
            await asyncio.sleep(interval)
        else:
            await working(1, task)
    # answer end


async def worker_2():
    while task_list2:
        task = task_list2.pop(0)
        if task.type != 2:
            task_list1.append(task)
            await asyncio.sleep(interval)
        else:
            await working(2, task)
    # answer end


async def main():
    task1 = asyncio.create_task(worker_1())
    task2 = asyncio.create_task(worker_2())
    await task2
    await task1
    #await asyncio.gather(worker_1(), worker_2())
    
await main()

worker 1 is working on task 1 and cost 2 seconds!
worker 2 is working on task 2 and cost 1 seconds!
worker 2 is working on task 2 and cost 3 seconds!
worker 1 is working on task 1 and cost 3 seconds!
worker 2 is working on task 2 and cost 4 seconds!
worker 1 is working on task 3 and cost 4 seconds!
worker 2 is working on task 2 and cost 2 seconds!
worker 1 is working on task 3 and cost 2 seconds!
